# Test notebook

__A. J. Tropiano [tropiano.4@osu.edu]__<br/>
__March 17, 2022__

The purpose of this notebook is for testing out code and/or scripts.

_Last update: March 25, 2022_

In [1]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

In [2]:
# Imports from A.T. codes
import densities
import momentum_distributions
from vnn import Potential
# You could import modules but I want a list of scripts here
# Check these off the list as you go through Jupyter notebooks generating
# figures
# import dmd, snmd, pmd
import modules.a2
import modules.figure_graphics
import modules.figure_labels
import modules.fourier_transform
import modules.integration
import modules.long_distance_operators
from modules.magnus import Magnus_split
import modules.momentum_projection_operator
import modules.phase_shifts
import modules.sdt
from modules.srg import get_transformation
import modules.tools
import modules.wave_function
# add missing ones

For now let's illustrate the overhaul by testing scripts and showing how to use them.

### Test `vnn.py`

In [3]:
# kvnn = 900
# channel = '3S1'
# kmax, kmid, ntot = 30.0, 4.0, 120

In [4]:
# # For comparing to old code
# from os import getcwd, chdir

# cwd = getcwd()
# old_code_dir = '../../../tropiano-srg-old'

# chdir(old_code_dir)

# from run_srg import run_srg as rs_old
# from potentials.vsrg_macos import vnn as vnn_old

# chdir(cwd)

In [5]:
# # __init__ (check)

# potential = Potential(kvnn, channel, kmax, kmid, ntot)
# print(potential.kvnn, potential.coupled_channel_bool, 
#       potential.potential_directory)

In [6]:
# # load_mesh (check)

# k_array, k_weights = potential.load_mesh()
# for ik, iw in zip(k_array, k_weights):
#     print(ik, iw)

In [7]:
# # load_potential (check)

# V_matrix = potential.load_potential()
# print(V_matrix)

In [8]:
# # load_hamiltonian (check)

# H_matrix = potential.load_hamiltonian()
# print(H_matrix)

In [9]:
# # Compare to Hamiltonian using vnn.py from tropiano-srg-old (check)

# chdir(old_code_dir)

# H_matrix_old = vnn_old.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
# print(H_matrix_old)

# chdir(cwd)

### Test `srg.py`

In [10]:
# # run_srg for Wegner (check)

# generator = 'Wegner'
# lambda_array = np.array([6.0, 3.0])

# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [11]:
# chdir(old_code_dir)

# d_old = rs_old(kvnn, channel, kmax, kmid, ntot, generator, lambda_array,
#                save=False)
# print(d_old[3.0])

# chdir(cwd)

In [12]:
# # run_srg for T (check)

# generator = 'T'

# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [13]:
# chdir(old_code_dir)

# d_old = rs_old(kvnn, channel, kmax, kmid, ntot, generator, lambda_array,
#                save=False)
# print(d_old[3.0])

# chdir(cwd)

In [14]:
# # run_srg for Block-diag (check)

# generator = 'Block-diag'
# lambda_bd_array = np.array([6.0, 3.0])
# lambda_array = np.array([1.0])

# d = potential.run_srg(generator, lambda_array, lambda_bd_array, save=False)
# print(d[3.0][1.0])

In [15]:
# chdir(old_code_dir)

# d_old = rs_old(kvnn, channel, kmax, kmid, ntot, generator, lambda_array,
#                lambda_bd=lambda_bd_array[-1], save=False)
# print(d_old[1.0])

# chdir(cwd)

In [16]:
# # compare evolved Hamiltonian using get_transformation (check)

# H_initial = potential.load_hamiltonian()
# H_evolved = potential.load_hamiltonian('srg', 'Wegner', 3.0)
# U_matrix = get_transformation(H_initial, H_evolved)
# print(U_matrix @ H_initial @ U_matrix.T / potential.hbar_sq_over_m)

### Test `magnus.py`

In [17]:
# Might be smart to test a really soft potential here for speed
kvnn = 111
channel = '1S0'
kmax, kmid, ntot = 10.0, 2.0, 120

potential = Potential(kvnn, channel, kmax, kmid, ntot)

In [18]:
# Do default Magnus arguments
k_max = 6
ds = 1e-5

In [19]:
# run_magnus for Wegner (check)

generator = 'Wegner'
lambda_array = np.array([6.0, 3.0])

d = potential.run_magnus(generator, lambda_array, k_max=k_max, ds=ds,
                         save=False)
print(d[3.0])

# run_srg for Wegner
d = potential.run_srg(generator, lambda_array, save=False)
print(d[3.0])

_____________________________________________________________________________________
Done evolving to final \lambda = 3 fm^-1 after 0.2598 minutes.
_____________________________________________________________________________________

Specifications:

kvnn = 111, channel = 1S0
kmax = 10.0, kmid = 2.0, ntot = 120
method = Magnus, generator = Wegner
k_max = 6, ds = 1.0e-05
[[ 6.22375956e-07 -1.22804364e-08 -3.77515109e-08 ...  1.39857228e-17
   1.06096159e-17  6.76337279e-18]
 [-1.22804364e-08  1.72026227e-05 -3.03134970e-07 ...  1.12299839e-16
   8.51909433e-17  5.43071523e-17]
 [-3.77515109e-08 -3.03134970e-07  1.03355206e-04 ...  3.45195338e-16
   2.61863961e-16  1.66931291e-16]
 ...
 [ 1.39857390e-17  1.12299945e-16  3.45195104e-16 ...  9.91800753e+01
  -3.67710342e-03 -2.39840883e-03]
 [ 1.06096101e-17  8.51909887e-17  2.61864126e-16 ... -3.67710342e-03
   9.96645753e+01 -1.92724981e-03]
 [ 6.76338024e-18  5.43071535e-17  1.66931162e-16 ... -2.39840883e-03
  -1.92724981e-03  9.9935

In [20]:
# # run_magnus for T (check)

# generator = 'T'
# lambda_array = np.array([6.0, 3.0])

# d = potential.run_magnus(generator, lambda_array, k_max=k_max, ds=ds,
#                          save=False)
# print(d[3.0])

# # run_srg for Wegner
# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [21]:
# # run_magnus for Block-diag

# generator = 'Block-diag'
# lambda_bd_array = np.array([6.0, 3.0])
# lambda_array = np.array([3.0])

# d = potential.run_magnus(generator, lambda_array, lambda_bd_array, 
#                          k_max=k_max, ds=ds, save=False)
# print(d[3.0][3.0])

# # run_srg for Block-diag (check)
# d = potential.run_srg(generator, lambda_array, lambda_bd_array, save=False)
# print(d[3.0][3.0])

In [22]:
# Test that Magnus_split at least works (check)

generator = 'Wegner'
lambda_array = np.array([6.0, 3.0])
k_max = 6
ds = 1e-5

magnus_split = Magnus_split(potential, generator)
d = magnus_split.magnus_split_evolve(lambda_array, k_max=k_max, ds=ds)
print(d[3.0])

[[ 6.22375956e-07 -1.22804364e-08 -3.77515109e-08 ...  9.54046070e-30
   5.61972023e-30  3.10939731e-30]
 [-1.22804364e-08  1.72026227e-05 -3.03134970e-07 ...  7.66101049e-29
   4.51264747e-29  2.49685275e-29]
 [-3.77515109e-08 -3.03134970e-07  1.03355206e-04 ...  2.35552883e-28
   1.38750253e-28  7.67706665e-29]
 ...
 [ 9.54046070e-30  7.66101049e-29  2.35552883e-28 ...  9.91800753e+01
  -3.67710342e-03 -2.39840883e-03]
 [ 5.61972023e-30  4.51264747e-29  1.38750253e-28 ... -3.67710342e-03
   9.96645753e+01 -1.92724981e-03]
 [ 3.10939731e-30  2.49685275e-29  7.67706665e-29 ... -2.39840883e-03
  -1.92724981e-03  9.99355553e+01]]


In [23]:
# Test figure_graphics.py

In [24]:
# Test figure_labels.py

In [25]:
# Test fourier_transform.py

In [26]:
# Test integration.py

In [27]:
# Test long_distance_operators.py

In [28]:
# Test momentum_projection_operator.py

In [29]:
# Test phase_shifts.py

In [30]:
# Test sdt.py

In [31]:
# Test srg.py

In [32]:
# Test tools.py

In [33]:
# Test wave_function.py